# Crawl điểm chuẩn 2024

In [1]:
# !pip install undetected-chromedriver selenium

In [1]:
import os
import time
import traceback
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException

In [ ]:
# os.chdir(r'D:\University\Applied Data Science\ADS_Final')

In [14]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

# ========== 2. TRUY CẬP TRANG ==========
target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)

# ✅ Thêm đoạn click "Xem thêm" để hiển thị đầy đủ nhóm ngành
try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

data = []
school_counter = 0

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group_index in range(len(groups)):
        groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
        group = groups[group_index]

        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
        time.sleep(1.5)
        try:
            action.move_to_element(group).click().perform()
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].click();", group)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-collapse-content")))
        time.sleep(1)

        group_name = group.find_element(By.TAG_NAME, "p").text.strip()
        majors = group.find_element(
            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
        ).find_elements(By.TAG_NAME, "a")

        for major_index in range(len(majors)):
            groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
            group = groups[group_index]
            action.move_to_element(group).click().perform()
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-collapse-content")))
            time.sleep(1)

            majors = group.find_element(
                By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
            ).find_elements(By.TAG_NAME, "a")

            try:
                major = majors[major_index]
                major_name = major.text.strip()
                major_link = major.get_attribute("href")
                print(f"[DEBUG] Đang xử lý ngành: {major_name}")
                driver.get(major_link)
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))

                # Click "Xem thêm" nhiều lần cho đến khi hết trường
                previous_count = 0
                while True:
                    school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
                    current_count = len(school_rows)
                    if current_count > previous_count:
                        previous_count = current_count
                        try:
                            see_more_button = WebDriverWait(driver, 3).until(
                                EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                            )
                            driver.execute_script("arguments[0].click();", see_more_button)
                            print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                            time.sleep(2)
                        except:
                            print("[DEBUG] Không còn nút 'Xem thêm', tiếp tục xử lý.")
                            break
                    else:
                        print(f"[DEBUG] Đã tải đủ tất cả {current_count} trường đào tạo.")
                        break

                # ✅ Thu thập danh sách các trường (text + link), không giữ element
                school_info = []
                school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
                for row in school_rows:
                    try:
                        cols = row.find_elements(By.TAG_NAME, "td")
                        if len(cols) >= 5:
                            name = cols[1].text.strip()
                            link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                            school_info.append((name, link))
                    except:
                        continue

                # ✅ Xử lý từng trường
                for idx, (school_name, school_link) in enumerate(school_info, 1):
                    print(f"[DEBUG]      ({idx}) Trường: {school_name}")
                    print(f"[DEBUG]        → Mở link trường: {school_link}")

                    try:
                        driver.get(school_link)
                        wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                        table = driver.find_element(By.ID, "diem-thi-thpt")
                        rows = table.find_elements(By.XPATH, ".//tbody/tr")
                        print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")

                        for row in rows:
                            cols = row.find_elements(By.TAG_NAME, "td")
                            if len(cols) >= 6:
                                data.append({
                                    "Nhóm ngành": group_name,
                                    "Ngành": major_name,
                                    "Trường đào tạo": school_name,
                                    "Mã ngành": cols[1].text.strip(),
                                    "Tên ngành": cols[2].text.strip(),
                                    "Tổ hợp môn": cols[3].text.strip(),
                                    "Điểm chuẩn 2024": cols[4].text.strip(),
                                    "Ghi chú": cols[5].text.strip()
                                })
                                school_counter += 1

                        if school_counter % 10 == 0:
                            print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")

                    except Exception:
                        print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                        print(traceback.format_exc())

                    # Quay lại trang ngành
                    driver.get(major_link)
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))
                    time.sleep(1)

            except Exception:
                print(f"[ERROR] ❌ Lỗi khi xử lý ngành.")
                print(traceback.format_exc())

except Exception as e:
    print("❌ Lỗi tổng thể:", e)
    print(traceback.format_exc())

finally:
    driver.quit()
    df = pd.DataFrame(data)
    if not df.empty:
        df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
        os.makedirs("Data", exist_ok=True)
        df.to_csv("Data/nganh_dao_tao.csv", index=False, encoding="utf-8-sig")
        print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao.csv")
    else:
        print("⚠️ Không có dữ liệu nào được lưu.")

[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] Đang xử lý ngành: Kế toán
[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: 142
[DEBUG] Đã tải đủ tất cả 142 trường đào tạo.
[DEBUG]      (1) Trường: Đại Học Kinh Tế Quốc Dân
[DEBUG]        → Mở link trường: https://diemthi.tuyensinh247.com/nganh-dao-tao/ke-toan-1.html?truong=KHA
[DEBUG]         Số dòng điểm tìm thấy: 2
[DEBUG]      (2) Trường: Đại Học Bách Khoa Hà Nội
[DEBUG]        → Mở link trường: https://diemthi.tuyensinh247.com/nganh-dao-tao/ke-toan-1.html?truong=BKA
[DEBUG]         Số dòng điểm tìm thấy: 1
[DEBUG]      (3) Trường: Học Viện Tài chính
[DEBUG]        → Mở link trường: https://diemthi.tuyensinh247.com/nganh-dao-tao/ke-toan-1.html?truong=HTC
[DEBUG]         Số dòng điểm tìm thấy: 2
[DEBUG]      (4) Trường: Học Viện Ngân Hàng
[DEBUG]        → Mở link trường: https://diemthi.tuyensinh247.com/nganh-dao-tao/ke-toan-1.html?truon

In [15]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

# ========== 2. TRUY CẬP TRANG ==========
target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)

# ✅ Click "Xem thêm" để hiển thị đầy đủ nhóm ngành
try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

# ✅ Thu thập danh sách tất cả các ngành trong từng nhóm ngành
group_data = []
try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group in groups:
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
        time.sleep(1)
        try:
            action.move_to_element(group).click().perform()
        except:
            driver.execute_script("arguments[0].click();", group)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-collapse-content")))
        time.sleep(1)

        group_name = group.find_element(By.TAG_NAME, "p").text.strip()
        majors = group.find_element(
            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
        ).find_elements(By.TAG_NAME, "a")

        for major in majors:
            major_name = major.text.strip()
            major_link = major.get_attribute("href")
            group_data.append((group_name, major_name, major_link))

except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành:", e)
    print(traceback.format_exc())

# ========== 3. TIẾN HÀNH CRAWL TỪ DANH SÁCH group_data ==========
data = []
school_counter = 0
for group_name, major_name, major_link in group_data:
    try:
        print(f"[DEBUG] Đang xử lý ngành: {major_name}")
        driver.get(major_link)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))

        # Click 'Xem thêm' nhiều lần nếu cần
        previous_count = 0
        while True:
            school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
            current_count = len(school_rows)
            if current_count > previous_count:
                previous_count = current_count
                try:
                    see_more_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                    )
                    driver.execute_script("arguments[0].click();", see_more_button)
                    print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                    time.sleep(2)
                except:
                    print("[DEBUG] Không còn nút 'Xem thêm', tiếp tục xử lý.")
                    break
            else:
                break

        school_info = []
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        for row in school_rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) >= 5:
                    name = cols[1].text.strip()
                    link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                    school_info.append((name, link))
            except:
                continue

        for idx, (school_name, school_link) in enumerate(school_info, 1):
            print(f"[DEBUG]      ({idx}) Trường: {school_name}")
            print(f"[DEBUG]        → Mở link trường: {school_link}")
            try:
                driver.get(school_link)
                wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                table = driver.find_element(By.ID, "diem-thi-thpt")
                rows = table.find_elements(By.XPATH, ".//tbody/tr")

                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if len(cols) >= 6:
                        data.append({
                            "Nhóm ngành": group_name,
                            "Ngành": major_name,
                            "Trường đào tạo": school_name,
                            "Mã ngành": cols[1].text.strip(),
                            "Tên ngành": cols[2].text.strip(),
                            "Tổ hợp môn": cols[3].text.strip(),
                            "Điểm chuẩn 2024": cols[4].text.strip(),
                            "Ghi chú": cols[5].text.strip()
                        })
                        school_counter += 1

                if school_counter % 10 == 0:
                    print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")

            except Exception:
                print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                print(traceback.format_exc())

            driver.get(major_link)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))
            time.sleep(1)

    except Exception:
        print(f"[ERROR] ❌ Lỗi khi xử lý ngành: {major_name}")
        print(traceback.format_exc())

# ========== 4. LƯU FILE ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv("Data/nganh_dao_tao_2.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao_2.csv")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")


[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "ant-collapse-content")]"}
  (Session info: chrome=136.0.7103.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0036FBD3+61635]
	GetHandleVerifier [0x0036FC14+61700]
	(No symbol) [0x001905D3]
	(No symbol) [0x001D899E]
	(No symbol) [0x001D8D3B]
	(No symbol) [0x001CE111]
	(No symbol) [0x001FD2E4]
	(No symbol) [0x001CE034]
	(No symbol) [0x001FD514]
	(No symbol) [0x0021E61B]
	(No symbol) [0x001FD096]
	(No symbol) [0x001CC840]
	(No symbol) [0x001CD6A4]
	GetHandleVerifier [0x005B7043+2450739]
	GetHandleVerifier [0x005B27C6+2432182]
	GetHandleVerifier [0x005CD50E+2542078]
	G

In [3]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

# ========== 2. TRUY CẬP TRANG ==========
target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)

# ✅ Click "Xem thêm" để hiển thị đầy đủ nhóm ngành
try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

# ========== 3. THU THẬP DANH SÁCH NGÀNH ==========

group_data = []

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group_index in range(len(groups)):
        try:
            groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
            group = groups[group_index]

            # Cuộn vào giữa màn hình để đảm bảo hiển thị
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
            time.sleep(1)

            # Thử tối đa 3 lần để mở nhóm ngành và lấy danh sách ngành
            success = False
            for attempt in range(3):
                try:
                    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                    group = groups[group_index]
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                    time.sleep(1.2)

                    try:
                        action.move_to_element(group).click().perform()
                    except:
                        driver.execute_script("arguments[0].click();", group)

                    # ✅ Chờ aria-expanded là true
                    WebDriverWait(driver, 5).until(
                        lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                    )

                    # ✅ Chờ có ít nhất một thẻ a trong collapse
                    WebDriverWait(driver, 5).until(
                        lambda d: len(group.find_element(
                            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                        ).find_elements(By.TAG_NAME, "a")) > 0
                    )

                    group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                    majors = group.find_element(
                        By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                    ).find_elements(By.TAG_NAME, "a")

                    print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")

                    for major in majors:
                        major_name = major.text.strip()
                        major_link = major.get_attribute("href")
                        group_data.append((group_name, major_name, major_link))

                    success = True
                    break  # ✅ Thành công, thoát vòng retry

                except Exception:
                    print(f"[WARN] Thử lần {attempt+1} thất bại khi mở nhóm ngành index {group_index}")
                    time.sleep(1)

            if not success:
                print(f"[ERROR] ❌ Không thể mở nhóm ngành tại index {group_index} sau 3 lần thử.")


            # Lấy group name và danh sách ngành
            group_name = group.find_element(By.TAG_NAME, "p").text.strip()
            majors = group.find_element(
                By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
            ).find_elements(By.TAG_NAME, "a")

            print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")

            for major in majors:
                major_name = major.text.strip()
                major_link = major.get_attribute("href")
                group_data.append((group_name, major_name, major_link))

        except Exception as e:
            print(f"[WARN] Không thể xử lý nhóm ngành tại index {group_index}")
            print(traceback.format_exc())

except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành:", e)
    print(traceback.format_exc())


# ========== 4. TIẾN HÀNH CRAWL TỪ DANH SÁCH ==========
data = []
school_counter = 0
for group_name, major_name, major_link in group_data:
    try:
        print(f"[DEBUG] Đang xử lý ngành: {major_name}")
        driver.get(major_link)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))

        # Click "Xem thêm" nhiều lần nếu có
        previous_count = 0
        while True:
            school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
            current_count = len(school_rows)
            if current_count > previous_count:
                previous_count = current_count
                try:
                    see_more_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                    )
                    driver.execute_script("arguments[0].click();", see_more_button)
                    print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                    time.sleep(2)
                except:
                    print("[DEBUG] Không còn nút 'Xem thêm'.")
                    break
            else:
                break

        # ✅ Thu thập các trường
        school_info = []
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        for row in school_rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) >= 5:
                    name = cols[1].text.strip()
                    link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                    school_info.append((name, link))
            except:
                continue

        # ✅ Truy cập từng trường và lấy bảng điểm
        for idx, (school_name, school_link) in enumerate(school_info, 1):
            print(f"[DEBUG]      ({idx}) Trường: {school_name}")
            print(f"[DEBUG]        → Mở link trường: {school_link}")
            try:
                driver.get(school_link)
                wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                table = driver.find_element(By.ID, "diem-thi-thpt")
                rows = table.find_elements(By.XPATH, ".//tbody/tr")
                print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")

                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if len(cols) >= 6:
                        data.append({
                            "Nhóm ngành": group_name,
                            "Ngành": major_name,
                            "Trường đào tạo": school_name,
                            "Mã ngành": cols[1].text.strip(),
                            "Tên ngành": cols[2].text.strip(),
                            "Tổ hợp môn": cols[3].text.strip(),
                            "Điểm chuẩn 2024": cols[4].text.strip(),
                            "Ghi chú": cols[5].text.strip()
                        })
                        school_counter += 1

                if school_counter % 10 == 0:
                    print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")

            except Exception:
                print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                print(traceback.format_exc())

    except Exception:
        print(f"[ERROR] ❌ Lỗi khi xử lý ngành: {major_name}")
        print(traceback.format_exc())

# ========== 5. LƯU FILE ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv("Data/nganh_dao_tao_3.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao_3.csv")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")


C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[WARN] Thử lần 1 thất bại khi mở nhóm ngành index 1
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[WARN] Thử lần 1 thất bại khi mở nhóm ngành index 3
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 3

KeyboardInterrupt: 

In [6]:
# ========== HÀM TẮT QUẢNG CÁO ==========
def close_ads_if_present(driver, wait, max_wait=2):
    try:
        ad_close = WebDriverWait(driver, max_wait).until(
            EC.element_to_be_clickable((
                By.XPATH, "//div[contains(@class, 'ant-modal')]//button[contains(@class, 'ant-modal-close')]"
            ))
        )
        driver.execute_script("arguments[0].click();", ad_close)
        print("[INFO] ❌ Đã tự động tắt popup quảng cáo.")
        time.sleep(1)
    except:
        pass

In [7]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

# ========== 2. TRUY CẬP TRANG ==========
target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)
close_ads_if_present(driver, wait)

try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

# ========== 3. THU THẬP DANH SÁCH NGÀNH ==========
group_data = []
printed_group_names = set()
seen_major_links = set()

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group_index in range(len(groups)):
        try:
            groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
            group = groups[group_index]
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
            time.sleep(1)

            success = False
            for attempt in range(3):
                try:
                    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                    group = groups[group_index]
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                    time.sleep(1.2)

                    try:
                        action.move_to_element(group).click().perform()
                    except:
                        driver.execute_script("arguments[0].click();", group)

                    WebDriverWait(driver, 5).until(
                        lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                    )

                    WebDriverWait(driver, 5).until(
                        lambda d: len(group.find_element(
                            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                        ).find_elements(By.TAG_NAME, "a")) > 0
                    )

                    group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                    majors = group.find_element(
                        By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                    ).find_elements(By.TAG_NAME, "a")

                    if group_name not in printed_group_names:
                        print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")
                        printed_group_names.add(group_name)

                    for major in majors:
                        major_name = major.text.strip()
                        major_link = major.get_attribute("href")
                        if major_link not in seen_major_links:
                            seen_major_links.add(major_link)
                            group_data.append((group_name, major_name, major_link))

                    success = True
                    break
                except Exception:
                    print(f"[WARN] Thử lần {attempt+1} thất bại khi mở nhóm ngành index {group_index}")
                    time.sleep(1)

            if not success:
                print(f"[ERROR] ❌ Không thể mở nhóm ngành tại index {group_index} sau 3 lần thử.")
        except Exception as e:
            print(f"[WARN] Không thể xử lý nhóm ngành tại index {group_index}")
            print(traceback.format_exc())

except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành:", e)
    print(traceback.format_exc())

# ========== 4. TIẾN HÀNH CRAWL TỪ DANH SÁCH ==========
data = []
school_counter = 0

for idx, (group_name, major_name, major_link) in enumerate(group_data, 1):
    try:
        print(f"[DEBUG] ({idx}/{len(group_data)}) Đang xử lý ngành: {major_name}")

        for try_load in range(2):
            try:
                driver.set_page_load_timeout(30)
                driver.get(major_link)
                close_ads_if_present(driver, wait)
                break
            except Exception as e:
                print(f"[WARN] ⚠️ Retry load ngành {major_name} lần {try_load+1}: {e}")
                if try_load == 1:
                    raise e

        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))

        previous_count = 0
        while True:
            school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
            current_count = len(school_rows)
            if current_count > previous_count:
                previous_count = current_count
                try:
                    see_more_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                    )
                    driver.execute_script("arguments[0].click();", see_more_button)
                    print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                    time.sleep(2)
                except:
                    print("[DEBUG] Không còn nút 'Xem thêm'.")
                    break
            else:
                break

        school_info = []
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        for row in school_rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) >= 5:
                    name = cols[1].text.strip()
                    link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                    school_info.append((name, link))
            except:
                continue

        for i, (school_name, school_link) in enumerate(school_info, 1):
            print(f"[DEBUG]      ({i}) Trường: {school_name}")
            print(f"[DEBUG]        → Mở link trường: {school_link}")

            for retry in range(2):
                try:
                    driver.set_page_load_timeout(30)
                    driver.get(school_link)
                    close_ads_if_present(driver, wait)
                    wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                    table = driver.find_element(By.ID, "diem-thi-thpt")
                    rows = table.find_elements(By.XPATH, ".//tbody/tr")

                    print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")

                    for row in rows:
                        cols = row.find_elements(By.TAG_NAME, "td")
                        if len(cols) >= 6:
                            data.append({
                                "Nhóm ngành": group_name,
                                "Ngành": major_name,
                                "Trường đào tạo": school_name,
                                "Mã ngành": cols[1].text.strip(),
                                "Tên ngành": cols[2].text.strip(),
                                "Tổ hợp môn": cols[3].text.strip(),
                                "Điểm chuẩn 2024": cols[4].text.strip(),
                                "Ghi chú": cols[5].text.strip()
                            })
                            school_counter += 1

                    if school_counter % 10 == 0:
                        print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")
                    break
                except Exception:
                    if retry == 1:
                        print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                        print(traceback.format_exc())
                    else:
                        print(f"[WARN] ⚠️ Thử lại lần 2...")

    except Exception:
        print(f"[ERROR] ❌ Lỗi khi xử lý ngành: {major_name}")
        print(traceback.format_exc())

# ========== 5. LƯU FILE ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv("Data/nganh_dao_tao_4.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao_4.csv")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")

C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[DEBUG] → Nhóm ngành: Y - Dược có 20 ngành
[DEBUG] → Nhóm ngành: Bác sĩ thú y có 3 ngành
[DEBUG] → Nhóm ngành: Công an - Quân đội có 30 ngành
[DEBUG] → Nhóm ngành: Thiết kế đồ họa - Game - Đa phương tiện có 5 ngành
[DEBUG] → Nhóm ngành: Xây dựng - Kiến trúc - Giao thông có 22 ngành
[DEBUG] → Nhóm ngành: Ngoại giao - Ngoại ngữ có 24 ngành
[DEBUG] → Nhóm ngành: Ngoại thương - Xuất nhập khẩu - Kinh Tế quốc tế có 9 ngành
[DEBUG] → Nhóm ngành: Du lịch-Khách sạn có 12 ngành
[DEBUG]

In [8]:
df

,Nhóm ngành,Ngành,Trường đào tạo,Mã ngành,Tên ngành,Tổ hợp môn,Điểm chuẩn 2024,Ghi chú
0,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,7340301,Kế toán,A00; A01; D01; D07,27.29,
1,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,EP04,Kế toán tích hợp chứng chi quốc tế (JCAEW CFAB),A00; A01; D01; D07,27.2,
2,Kế toán - Kiểm toán,Kế toán,Đại Học Bách Khoa Hà Nội,EM4,Kế toán,"A00; A01; D01; K01 (Toán, Văn, Lý/Hóa/Sinh/Tin)",25.8,
3,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301,Kế toán,A00; A01; D01; D07,26.45,
4,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301C21,Kế toán doanh nghiệp (theo định hướng ACCA),A01; D01; D07,34.35,"CT định hướng CCQT, Tiếng Anh hệ số 2"
...,...,...,...,...,...,...,...,...
6447,Khoa học tư nhiên khác,,Trường Đại Học Sư Phạm TPHCM,7440102,Vật lý học,A00; A01; D90,24.44,
6448,Khoa học tư nhiên khác,,Trường Đại Học Khoa Học Tự Nhiên TPHCM,7440102_DKD,Vật lý học (CT tăng cường tiếng Anh),"A00; A01; C01; C03; A04; A10; (Toán, Lí, Công ...",,
6450,Khoa học tư nhiên khác,,Trường Đại Học Khoa Học Tự Nhiên Hà Nội,QHT03,Vật lý học,A00; A01; A02; C01,25.3,
6451,Khoa học tư nhiên khác,,Trường Đại Học Khoa Học Huế,7440102,Vật lý học,"A00; A01; C01; (Toán, Lí, Tin)",,
